🟩 CELL 1 — Environment + Seed (you already know this)

In [ ]:
!pip install -q mne tensorflow
import os, random, numpy as np, tensorflow as tf, mne

SEED = 42
np.random.seed(SEED)
random.seed(SEED)
tf.random.set_seed(SEED)

print("NumPy:", np.__version__)
print("MNE:", mne.__version__)
print("TF:", tf.__version__)

NumPy: 2.0.2
MNE: 1.11.0
TF: 2.19.0


🟩 CELL 2 — Mount Drive + Extract Dataset

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

import zipfile, os

zip_path = "/content/drive/MyDrive/EEG_PD_Project/UCSD_PD_EEG.zip"
extract_path = "/content/ds002778"

os.makedirs(extract_path, exist_ok=True)
with zipfile.ZipFile(zip_path, "r") as z:
    z.extractall(extract_path)

print("Dataset extracted")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Dataset extracted


🟩 CELL 3 — Define BIDS Root + Load Participants

In [ ]:
import pandas as pd

BIDS_ROOT = "/content/ds002778/ds002778-download"
participants = pd.read_csv(
    os.path.join(BIDS_ROOT, "participants.tsv"),
    sep="\t"
)

participants.head()

,participant_id,age,gender,hand,MMSE,NAART,disease_duration,rl_deficits,notes
0,sub-hc1,54,f,r,30,48,NaN,NaN,NaN
1,sub-hc2,50,f,r,30,55,NaN,NaN,NaN
2,sub-pd3,52,f,r,29,44,9.0,R,NaN
3,sub-hc4,50,f,r,30,52,NaN,NaN,NaN
4,sub-pd5,67,f,r,29,48,2.0,"No clear side, perhaps more L",NaN


🟩 CELL 4 — Create Labels (PD = 1, HC = 0)

In [ ]:
subject_labels = {}

for _, row in participants.iterrows():
    sid = row["participant_id"]
    if sid.startswith("sub-pd"):
        subject_labels[sid] = 1
    elif sid.startswith("sub-hc"):
        subject_labels[sid] = 0

print("Total subjects:", len(subject_labels))
print("PD:", sum(subject_labels.values()))
print("HC:", len(subject_labels) - sum(subject_labels.values()))

Total subjects: 31
PD: 15
HC: 16


🟩 CELL 5 — Collect EEG FILE PATHS (NO LOADING)

In [ ]:
eeg_files = []

for sub in sorted(os.listdir(BIDS_ROOT)):
    if not sub.startswith("sub-"):
        continue

    for ses in os.listdir(os.path.join(BIDS_ROOT, sub)):
        eeg_dir = os.path.join(BIDS_ROOT, sub, ses, "eeg")
        if not os.path.exists(eeg_dir):
            continue

        for f in os.listdir(eeg_dir):
            if f.endswith("_eeg.bdf"):
                eeg_files.append({
                    "subject": sub,
                    "path": os.path.join(eeg_dir, f),
                    "label": subject_labels[sub]
                })

print("EEG files found:", len(eeg_files))
print(eeg_files[0])

EEG files found: 46
{'subject': 'sub-hc1', 'path': '/content/ds002778/ds002778-download/sub-hc1/ses-hc/eeg/sub-hc1_ses-hc_task-rest_eeg.bdf', 'label': 0}


🟩 CELL 6 — WINDOWING + DATASET CREATION (THIS CREATES X, y, groups)


In [ ]:
X, y, groups = [], [], []

WINDOW_SEC = 3.0
OVERLAP = 0.5

def extract_windows(raw, window_sec, overlap):
    sf = raw.info["sfreq"]
    win = int(window_sec * sf)
    step = int(win * (1 - overlap))
    data = raw.get_data()
    return np.array([
        data[:, i:i+win]
        for i in range(0, data.shape[1] - win, step)
    ])

for entry in eeg_files:
    raw = mne.io.read_raw_bdf(entry["path"], preload=True, verbose=False)
    raw.pick_types(eeg=True)
    raw.set_eeg_reference("average", verbose=False)
    raw.filter(1., 40., verbose=False)

    windows = extract_windows(raw, WINDOW_SEC, OVERLAP)
    X.append(windows)
    y.extend([entry["label"]] * len(windows))
    groups.extend([entry["subject"]] * len(windows))

X = np.concatenate(X)
y = np.array(y)
groups = np.array(groups)

print("X:", X.shape)
print("y:", y.shape)
print("groups:", groups.shape)

NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).

🟩 CELL 7 — NOW (AND ONLY NOW) SAVE ARRAYS

In [ ]:
np.save("/content/drive/MyDrive/X.npy", X)
np.save("/content/drive/MyDrive/y.npy", y)
np.save("/content/drive/MyDrive/groups.npy", groups)

print("Saved X, y, groups")

Saved X, y, groups


In [ ]:
# Cell 8 — Reload X, y, groups from Drive (SAFE CHECKPOINT)

import numpy as np

X = np.load("/content/drive/MyDrive/X.npy", mmap_mode="r")
y = np.load("/content/drive/MyDrive/y.npy")
groups = np.load("/content/drive/MyDrive/groups.npy")

print("X shape:", X.shape)
print("y shape:", y.shape)
print("groups shape:", groups.shape)

# Basic sanity checks (NO learning yet)
print("Unique labels:", np.unique(y))
print("Unique subjects:", len(np.unique(groups)))

X shape: (5964, 40, 1536)
y shape: (5964,)
groups shape: (5964,)
Unique labels: [0 1]
Unique subjects: 31


In [ ]:
# Cell 9 — Reshape X for EEGNet (SAFE, FAST)

X_eegnet = X[..., np.newaxis]

print("EEGNet input shape:", X_eegnet.shape)

EEGNet input shape: (5964, 40, 1536, 1)


In [ ]:
# Cell 10 — EEGNet Architecture Definition (NO TRAINING)

from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Conv2D, DepthwiseConv2D, SeparableConv2D,
    BatchNormalization, Activation, AveragePooling2D,
    Dropout, Flatten, Dense
)
from tensorflow.keras.constraints import max_norm


def build_eegnet(n_channels, n_samples):
    inputs = Input(shape=(n_channels, n_samples, 1))

    # Block 1
    x = Conv2D(
        filters=8,
        kernel_size=(1, 64),
        padding="same",
        use_bias=False
    )(inputs)
    x = BatchNormalization()(x)

    x = DepthwiseConv2D(
        kernel_size=(n_channels, 1),
        depth_multiplier=2,
        use_bias=False,
        depthwise_constraint=max_norm(1.0)
    )(x)
    x = BatchNormalization()(x)
    x = Activation("elu")(x)
    x = AveragePooling2D((1, 4))(x)
    x = Dropout(0.5)(x)

    # Block 2
    x = SeparableConv2D(
        filters=16,
        kernel_size=(1, 16),
        padding="same",
        use_bias=False
    )(x)
    x = BatchNormalization()(x)
    x = Activation("elu")(x)
    x = AveragePooling2D((1, 8))(x)
    x = Dropout(0.5)(x)

    # Classifier
    x = Flatten()(x)
    outputs = Dense(1, activation="sigmoid")(x)

    model = Model(inputs, outputs)
    return model


# Sanity check (no training)
model = build_eegnet(
    n_channels=X_eegnet.shape[1],
    n_samples=X_eegnet.shape[2]
)
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 40, 1536, 1)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 40, 1536, 8)    │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 40, 1536, 8)    │            32 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ depthwise_conv2d                │ (None, 1, 1536, 16)    │           640 │
│ (DepthwiseConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 1, 1536, 16)    │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 1, 1536, 16)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d               │ (None, 1, 384, 16)     │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1, 384, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ separable_conv2d                │ (None, 1, 384, 16)     │           512 │
│ (SeparableConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 1, 384, 16)     │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 1, 384, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_1             │ (None, 1, 48, 16)      │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 1, 48, 16)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 768)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           769 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,593 (10.13 KB)

 Trainable params: 2,513 (9.82 KB)

 Non-trainable params: 80 (320.00 B)

In [ ]:
# Cell 11 — STRICT LOSO with Resume + Checkpointing (FIXED FOR KERAS)

import os
import numpy as np
import tensorflow as tf

# ===== Paths =====
BASE_DIR = "/content/drive/MyDrive/EEGNet_LOSO"
os.makedirs(BASE_DIR, exist_ok=True)

RESULTS_PATH = os.path.join(BASE_DIR, "loso_results.npy")
DONE_PATH = os.path.join(BASE_DIR, "completed_subjects.txt")

# ===== Resume state =====
if os.path.exists(DONE_PATH):
    with open(DONE_PATH, "r") as f:
        completed_subjects = set(line.strip() for line in f)
else:
    completed_subjects = set()

if os.path.exists(RESULTS_PATH):
    results = list(np.load(RESULTS_PATH, allow_pickle=True))
else:
    results = []

subjects = np.unique(groups)

print("Total subjects:", len(subjects))
print("Already completed:", len(completed_subjects))

# ===== LOSO Loop =====
for test_sub in subjects:

    if test_sub in completed_subjects:
        print(f"[SKIP] {test_sub} already done")
        continue

    print(f"\n[TRAINING] Test subject: {test_sub}")

    # STRICT SUBJECT SPLIT
    train_idx = groups != test_sub
    test_idx = groups == test_sub

    X_train = X_eegnet[train_idx]
    y_train = y[train_idx]
    X_test = X_eegnet[test_idx]
    y_test = y[test_idx]

    # Fresh model
    model = build_eegnet(
        n_channels=X_eegnet.shape[1],
        n_samples=X_eegnet.shape[2]
    )

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
        loss="binary_crossentropy"
    )

    # Correct checkpoint filename
    ckpt_path = os.path.join(BASE_DIR, f"{test_sub}.weights.h5")

    ckpt_cb = tf.keras.callbacks.ModelCheckpoint(
        ckpt_path,
        monitor="loss",
        save_best_only=True,
        save_weights_only=True,
        verbose=0
    )

    # Train
    model.fit(
        X_train,
        y_train,
        epochs=30,
        batch_size=64,
        verbose=0,
        callbacks=[ckpt_cb]
    )

    # Load best weights
    model.load_weights(ckpt_path)

    # Predict
    y_prob = model.predict(X_test, verbose=0).ravel()

    # Save raw outputs ONLY
    results.append({
        "subject": test_sub,
        "y_true": y_test,
        "y_prob": y_prob
    })

    # Immediate checkpoint
    np.save(RESULTS_PATH, results)

    with open(DONE_PATH, "a") as f:
        f.write(test_sub + "\n")

    completed_subjects.add(test_sub)

    print(f"[DONE] {test_sub} | windows: {len(y_test)}")

print("\nLOSO TRAINING COMPLETE")
print("Saved to:", RESULTS_PATH)

Total subjects: 31
Already completed: 0

[TRAINING] Test subject: sub-hc1
